In [2]:
#template matching in WiiPlay.mp4
import cv2
import numpy as np


#將滑鼠移動的軌跡繪製成方框
def mouse_callback(event, _x, _y, flags, param):
    global show_img, x0, y0, w, h, mouse_pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_pressed = True
        x0, y0 = _x, _y
        show_img = np.copy(img)
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_pressed:
            show_img = np.copy(img)
            cv2.rectangle(show_img, (x0, y0), (_x, _y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_pressed = False
        w, h = _x - x0, _y - y0




#讀取要用來標定臉部的影格
cap = cv2.VideoCapture('WiiPlay.mp4')
# Check if the video file is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open the video file")
frame_seq = 4820
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)

ret, img = cap.read()
if ret == False:
    exit()
height, width = img.shape[0:2]
img = cv2.resize(img, (int(width/2), int(height/2)))
#print(img.shape)
show_img = np.copy(img)
mouse_pressed = False
y0 = x0 = w = h = 0


#將影格傳入函數，繪製方框，並記錄標記被人臉的座標
cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)

while True:
    cv2.imshow('image', show_img)
    k = cv2.waitKey(1)
    if k == ord('a') and not mouse_pressed:
        if w*h > 0:
            break
cv2.destroyAllWindows()

#取出標記出的人臉區域
template = np.copy(img[y0:y0+h, x0:x0+w])
print(template.shape)

#設定辨識模式
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
show_img = np.copy(img)
index = 1 #給不同模式 index


while True:
    # while 只設定 4820~5000 影格之間
    frame_seq += 1
    if frame_seq > 5000:
        frame_seq = 4820
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)

    ret, img = cap.read()
    if ret == False:
        exit()
  
    img = cv2.resize(img, (int(width/2), int(height/2)))

    if k == 27:
        break

    #做辨識模式的切換設定
    else:
        if k > 0 and chr(k).isdigit():
            index0 = int(chr(k))
            if 0 <= index0 < len(methods):
                index = index0
        method = methods[index]

        
        #進行辨識
        res = cv2.matchTemplate(img, template, eval(method))
        #print('min:{} max:{}'.format(np.min(res), np.max(res)))
        res = cv2.normalize(res, None, 0, 1, cv2.NORM_MINMAX)            
        #print('min_n:{} max_n:{}'.format(np.min(res), np.max(res)))
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        res_img = np.copy(res) 
        res_img = cv2.resize(res_img, (int(width/2), int(height/2)))*255
        res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR).astype(np.uint8)

        #使用選定的辨識模式，並在辨識到的位置繪製方框
        if (index >= methods.index('cv2.TM_SQDIFF')):            
            cv2.rectangle(res_img, (min_loc[0] - int(w/2), min_loc[1] - int(h/2)), (min_loc[0] + int(w/2), min_loc[1] + int(h/2)),(0, 0, 255), 2)
            cv2.rectangle(show_img, (min_loc[0] - int(w/2), min_loc[1] - int(h/2)), (min_loc[0] + int(w/2), min_loc[1] + int(h/2)),(0, 0, 255), 2)
        
        #使用選定的辨識模式，並在辨識到的位置繪製方框
        elif (index < methods.index('cv2.TM_SQDIFF')):            
            cv2.rectangle(res_img, (max_loc[0] - int(w/2), max_loc[1] - int(h/2)), (max_loc[0] + int(w/2), max_loc[1] + int(h/2)),(0, 0, 255), 2)
            cv2.rectangle(show_img, (max_loc[0] - int(w/2), max_loc[1] - int(h/2)), (max_loc[0] + int(w/2), max_loc[1] + int(h/2)),(0, 0, 255), 2)
        
        
        # for y in range(0, res.shape[0]):
        #     for x in range(0, res.shape[1]):
        #         if (index >= methods.index('cv2.TM_SQDIFF')) and (res[y,x] < 0.01):            
        #             cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 1)
        #         elif (index < methods.index('cv2.TM_SQDIFF')) and (res[y,x] > 0.99):            
        #             cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 1)
        
                
        cv2.putText(res_img, method, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # show_img = np.hstack((img, res_img))

        cv2.imshow('image', img)
        cv2.imshow('method', res_img)


        k = cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

(64, 53, 3)
